# Notebook Downloading Mars Seismic Data, Splitting Windows
This notebook downloads mseed data from IRIS and splits it into 12 second windows sliding by 3 seconds

In [ ]:
%config IPCompleter.greedy=True
%matplotlib inline

import csv, json, os, sys
import requests
from time import sleep

import obspy
from obspy import read

from obspy.core.event import read_events
from obspy import read_inventory
from datetime import datetime, timedelta
from obspy.core import UTCDateTime


DATA_PATH = os.path.expanduser('~/Desktop/seisml-master/testing_ground/mars_insight_seismic/script_tests/')

## Download and Sample
the following:
* net=XB
* sta=ELYSE
* cha=BHU,BHV,BHW
* loc=02

In [ ]:
def download_data_availability(data_path=DATA_PATH):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass
    
    payload = {
        'option': 'com_ajax',
        'data': 'ELYSE',
        'format': 'json',
        'module': 'seis_data_available'
    }
    r = requests.post('https://www.seis-insight.eu/en/science/seis-data/seis-data-availability', payload)
    with open(os.path.join(data_path, 'data_availability.json'), 'wb') as f:
        f.write(r.content)

In [ ]:
download_data_availability()

In [ ]:
with open(os.path.join(DATA_PATH, 'data_availability.json'), 'r') as f:
    raw_ava = json.load(f)

In [ ]:
ava = []
for t in raw_ava['data']:
    if t['network'] == 'XB' and t['location'] == '02' and t['channel'] == 'BHU':
        ava.append(t)

In [ ]:
ava[0]

example data req: `http://ws.ipgp.fr/fdsnws/dataselect/1/query?network=XB&amp;station=ELYSE&amp;startTime=2019-02-12T02:43:01&amp;endTime=2019-02-12T04:14:38&amp;location=02&amp;channel=BH?&amp;nodata=404`

In [ ]:
def download_mseed(event, channel='BH?', data_path=DATA_PATH + 'raw/'):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass

    payload = {
        'network': event['network'],
        'station': event['station'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'location': event['location'],
        'channel': channel
    }
    
    req = requests.get('http://ws.ipgp.fr/fdsnws/dataselect/1/query', params=payload)
    file_name = '-'.join([event['network'], event['station'], event['location'], event['startTime'], event['endTime']]) + '.mseed'
    print('downloading: %s' % file_name)
    path = os.path.join(data_path, file_name)
    with open(path, 'wb') as c:
        c.write(req.content)
    return path

In [ ]:
# download all examples:
for event in ava:
    download_mseed(event)
    sleep(1)

In [ ]:
with open(os.path.join(DATA_PATH + 'raw/', 'catelog.json'), 'w') as f:
    json.dump(ava, f)

In [ ]:
file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir(DATA_PATH + 'raw/')))
print('num of files: %i' % len(file_names))

In [ ]:
streams = []
for file in file_names:
    streams.append(obspy.read(os.path.join(DATA_PATH + 'raw/', file)))

As some streams have more than 3 traces (in multiples of 3), we must go through each stream, cut each trace, and rejoin traces to their matching other 2 traces. This is accomplished in the following. 

This next box is a test for a stream with a small time frame in order to ensure the code works. 

In [ ]:
windows = []
sliced = []
for trace in streams[1]:
    start = UTCDateTime(trace.stats.starttime)
    end = UTCDateTime(trace.stats.endtime)
    while start + 12 < end:
        sliced.append(trace.slice(start,start+12))
        start = start+1.5
for aSlice in sliced:
    window = []
    window.append(aSlice)
    sliced.remove(aSlice)
    for bSlice in sliced:
        if bSlice.stats.starttime == aSlice.stats.starttime:
            window.append(bSlice)
            sliced.remove(bSlice)
    windows.append(obspy.core.Stream(window))


This following box is to run for all streams. Takes a VERY long time so be prepared!

In [ ]:
windows = []
for stream in streams:
    sliced = []
    for trace in stream:
        start = UTCDateTime(trace.stats.starttime)
        end = UTCDateTime(trace.stats.endtime)
        while start + 12 < end:
            sliced.append(trace.slice(start,start+12))
            start = start+3
    for aSlice in sliced:
        window = []
        window.append(aSlice)
        sliced.remove(aSlice)
        for bSlice in sliced:
            if bSlice.stats.starttime == aSlice.stats.starttime:
                window.append(bSlice)
                sliced.remove(bSlice)
        windows.append(obspy.core.Stream(window))